# Загрузка сгенерированных похожих данных в Clickhouse

Для взаимодействия с Clickhouse будем использовать clickhouse-driver.

In [ ]:
!pip install clickhouse-driver -q

     |████████████████████████████████| 608 kB 5.3 MB/s 


Путь к файлу с данными на Google Drive.

In [ ]:
input_file_path = '/content/drive/MyDrive/Интеллектуальный анализ данных в бизнесе 2 Гурьянов 11-909/Задача 2/fns_fiscal_generated_1M.csv'

In [ ]:
from google.colab import drive
drive.mount('drive')

Для удобства параметры подключения хранятся в файле, в формате json. Импортируем с Google Drive модуль, созданный мной. Он содержит функцию, извлекающую параметры подключения из файла в формате json и помещающую их в словарь, который можно напрямую распаковать в параметры конструктора клиента Clickhouse.

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Интеллектуальный анализ данных в бизнесе 2 Гурьянов 11-909/Задача 2")
from dfclient import connection_parameters_from_json

Создадим объект клиента Clickhouse.

In [ ]:
from clickhouse_driver import Client
client = Client(**connection_parameters_from_json("/content/drive/MyDrive/Интеллектуальный анализ данных в бизнесе 2 Гурьянов 11-909/Задача 2/ClickhouseConnectionParameters.json"))

Для проверки работоспособности клиента получим список баз данных и список таблиц.

In [ ]:
result = client.execute('SHOW databases')
result

[('INFORMATION_SCHEMA',), ('default',), ('information_schema',), ('system',)]

In [ ]:
result = client.execute('SHOW tables')
result

[('fns_fiscal',)]

Создадим таблицу, куда мы будем загружать данные. Структура таблицы соостветствует структуре данных.

In [ ]:
result = client.execute("DROP TABLE IF EXISTS fns_fiscal")

In [ ]:
# Типы чувствительны к регистру
result = client.execute("""
CREATE TABLE IF NOT EXISTS fns_fiscal (
    protocolVersion String,
    operationType String,
    shiftNumber UInt16,
    totalSum UInt32,
    ecashTotalSum UInt32,
    nds10 UInt32,
    nds18 UInt32,
    nds20 UInt32,
    dateTime date,
    requestNumber UInt16,
    taxationType String,
    receiptCode String,
    internetSign String,
    senderAddress String,
    buyerAddress String,
    userProperty_key String,
    userProperty_value String,
    sellerAddress String,
    fiscalDocumentFormatVer String,
    paymentAgentType String,
    propertiesUser_propertyName String,
    retailAddress String,
    code String,
    propertiesUser_key String,
    propertiesUser_value String
) ENGINE = MergeTree()
ORDER BY totalSum
""")
result

[]

Загрузим данные в созданную таблицу. Код загрузки данных в Clickhouse из файла формата csv адаптирован из кода в документации clickhouse-driver:

https://clickhouse-driver.readthedocs.io/en/latest/misc.html

In [ ]:
from math import floor

string_columns = ['protocolVersion', 'operationType', 'taxationType', 'receiptCode', 'internetSign', 'senderAddress', 'buyerAddress', 'userProperty_key', 'userProperty_value', 'sellerAddress',\
                  'fiscalDocumentFormatVer', 'paymentAgentType', 'propertiesUser_propertyName', 'retailAddress', 'code', 'propertiesUser_key', 'propertiesUser_value']

int_columns = ['shiftNumber', 'totalSum', 'ecashTotalSum', 'nds10', 'nds18', 'nds20', 'requestNumber']

converters = {}

for column in string_columns:
    converters[column] = str

for column in int_columns:
    converters[column] = lambda x: floor(float(x))

converters['dateTime'] = lambda x: datetime.strptime(x, '%Y-%m-%d')

In [ ]:
from csv import DictReader
from datetime import datetime

def iter_csv(filename, converters):
    with open(filename, 'r') as f:
        reader = DictReader(f)
        for line in reader:
            yield {k: (converters[k](v) if k in converters else v) for k, v in line.items()}

client.execute('INSERT INTO fns_fiscal VALUES', iter_csv(input_file_path, converters))

1000000

Проверим количество строк, содержащихся в таблице. Как и следовало ожидать, количество строк в таблице равно количеству строк в датасете.

In [ ]:
client.execute('SELECT count() FROM fns_fiscal')

[(1000000,)]